In [1]:
! ls ../../data/

CrowdFlowerAnnotations.txt  Flickr8k.token.txt	       machine_translation
ExpertAnnotations.txt	    Flickr_8k.devImages.txt    readme.txt
Flicker8k_smaller	    Flickr_8k.testImages.txt
Flickr8k.lemma.token.txt    Flickr_8k.trainImages.txt


In [2]:
! pip3 install jieba

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.2/19.2 MB 2.8 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
  Created wheel for jieba: filename=jieba-0.42.1-py3-none-any.whl size=19314477 sha256=ed27d19a84f32bfb6048db6e791987820e2c52697ae582f47ab1202f2dc61d03
  Stored in directory: /root/.cache/pip/wheels/24/aa/17/5bc7c72e9a37990a9620cc3aad0acad1564dcff6dbc2359de3
Successfully built jieba

[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [10]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, GRU, Dense, Layer
import numpy as np
import random
import jieba

batch_size = 16  
epochs = 35  
latent_dim = 128  # Latent dimensionality of the encoding space.
num_samples = 2000  # Number of samples to train on.
data_path = '../../data/machine_translation/cmn.txt'

input_texts = []
target_texts = []
input_tokens = set()
target_tokens = set()
with open(data_path, 'r', encoding='utf-8') as f:
    lines = f.read().split('\n')

# for line in lines[: min(num_samples, len(lines) - 1)]:
for line in random.sample(lines, num_samples):
    input_text, target_text = line.split('\t')
    
    # We use "\t" as the "start sequence" and "\n" as "end sequence"
    target_text = '\t' + target_text + '\n'
    
    tmp = []
    for token in input_text.split(" "):
        token = token.replace(",", "").replace(".", "").replace("?", "").lower()
        tmp.append(token)
        if token not in input_tokens:
            input_tokens.add(token)
    input_texts.append(tmp)
    
    tmp = []
    for token in jieba.cut(target_text, cut_all=False):
        tmp.append(token)
        if token not in target_tokens:
            target_tokens.add(token)
    target_texts.append(tmp)

input_tokens = list(input_tokens)
target_tokens = list(target_tokens)
num_encoder_tokens = len(input_tokens)
num_decoder_tokens = len(target_tokens)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

print('Number of samples:', len(input_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)

print("\ninput data set:")
print(input_texts[:10])
print("\ntarget data set:")
print(target_texts[:10])

Number of samples: 2000
Number of unique input tokens: 2185
Number of unique output tokens: 3223
Max sequence length for inputs: 17
Max sequence length for outputs: 22

input data set:
[['a', 'string', 'of', 'disasters', 'struck', 'the', 'region'], ['tom', 'has', 'been', 'under', 'a', 'great', 'deal', 'of', 'strain', 'lately'], ['tom', 'was', 'questioned', 'by', 'the', 'police'], ['i', "can't", 'keep', 'up', 'with', 'tom'], ['do', 'what', 'you', 'think', 'is', 'right'], ['this', 'is', 'all', 'i', 'know'], ['what', 'tom', 'said', "doesn't", 'apply', 'in', 'this', 'situation'], ['he', 'differs', 'from', 'his', 'brother'], ['i', 'prefer', 'milk', 'to', 'juice'], ['it', 'is', 'wrong', 'to', 'cheat', 'at', 'cards']]

target data set:
[['\t', '一連串', '的', '災難', '襲擊', '這個', '地區', '。', '\n'], ['\t', '汤姆', '最近', '压力', '特别', '大', '。', '\n'], ['\t', '汤姆', '被', '警察', '盘问', '。', '\n'], ['\t', '我', '跟不上', '汤姆', '。', '\n'], ['\t', '做', '你', '認為', '正確', '的', '事', '。', '\n'], ['\t', '這是', '我', '所', '知道'

In [11]:
input_token_index = dict(
    [(token, i) for i, token in enumerate(input_tokens)])
target_token_index = dict(
    [(token, i) for i, token in enumerate(target_tokens)])

print("\ninput index:")
print(input_token_index)

print("\ntarget index:")
print(target_token_index)


input index:
{'control': 0, 'boy': 1, 'sunday': 2, 'confidence': 3, 'himself': 4, 'list': 5, "devil's": 6, 'numbers': 7, 'finished': 8, 'hoped': 9, 'brochure': 10, 'language': 11, 'apple': 12, 'should': 13, 'optimistic': 14, "how're": 15, 'mathematical': 16, 'duty-free': 17, 'given': 18, 'flights': 19, 'communicate': 20, 'army': 21, '25': 22, 'york': 23, 'satellite': 24, 'tears': 25, 'thief': 26, 'doll': 27, 'traffic': 28, 'ambiguous': 29, 'covered': 30, "minutes'": 31, 'honest': 32, 'slight': 33, 'show': 34, 'gain': 35, 'yet': 36, 'danger': 37, 'extremely': 38, 'leave': 39, 'solve': 40, 'rainy': 41, 'hunting': 42, 'side': 43, "i'm": 44, 'louder': 45, 'soldier': 46, 'parliament': 47, 'live': 48, 'loses': 49, 'whether': 50, 'disasters': 51, 'share': 52, 'reading': 53, 'stopped': 54, 'became': 55, 'return': 56, 'cake': 57, 'closed': 58, 'come': 59, 'times': 60, 'ball': 61, 'empty': 62, 'hours': 63, 'coach': 64, 'suitable': 65, 'stolen': 66, 'games': 67, 'choices': 68, 'could': 69, 'anxi

In [12]:
encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens),
    dtype='float32')
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')

for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, token in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[token]] = 1.
    for t, token in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[token]] = 1.
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[token]] = 1.

print("The shape is (batch_size, time_steps, input_dim)")
print("so it will be (number of x, length of each x, dimension of x)")
print("\nencoder input shape:")
print(encoder_input_data.shape)
print("\ndecoder input shape:")
print(decoder_input_data.shape)
print("\ndecoder target shape:")
print(decoder_target_data.shape)
            
print("\nencoder input data (1-hot):")
print(encoder_input_data[-2:])
print("\ndecoder input data (1-hot):")
print(decoder_input_data[-2:])
print("\ndecoder target data (1-hot):")
print(decoder_target_data[-2:])

The shape is (batch_size, time_steps, input_dim)
so it will be (number of x, length of each x, dimension of x)

encoder input shape:
(2000, 17, 2185)

decoder input shape:
(2000, 22, 3223)

decoder target shape:
(2000, 22, 3223)

encoder input data (1-hot):
[[[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]]

decoder input data (1-hot):
[[[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]]

decoder target data (1-hot):
[[[0. 0. 0. ... 0. 0. 0.]
 

In [13]:
class Encoder(Model):
    def __init__(self, vocab_size, latent_dim, batch_sz):
        super(Encoder, self).__init__()
        self.batch_sz = batch_sz
        self.latent_dim = latent_dim
        self.lstm = GRU(self.latent_dim,
                        return_sequences=True,
                        return_state=True,
                        recurrent_initializer='glorot_uniform')

    def call(self, x):
        output, state = self.lstm(x)
        return output, state


encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = Encoder(num_encoder_tokens, latent_dim, batch_size)
encoder_outputs, state = encoder(encoder_inputs)

print(f"encoder_inputs: {encoder_inputs.shape}")
print(f"state: {state.shape}")
print(f"encoder_outputs: {encoder_outputs.shape}")

encoder_inputs: (None, None, 2185)
state: (None, 128)
encoder_outputs: (None, None, 128)


In [27]:
class Decoder(Model):
    def __init__(self, vocab_size, latent_dim, batch_sz):
        super(Decoder, self).__init__()
        self.batch_sz = batch_sz
        self.latent_dim = latent_dim
        self.lstm = GRU(latent_dim,
                        return_sequences=True,
                        return_state=True,
                        recurrent_initializer='glorot_uniform')
        self.fc = Dense(vocab_size)

    def call(self, x, hidden):
        # context_vector, attention_weights = self.attention(hidden, enc_output)
        # x = tf.concat([tf.expand_dims(context_vector, 1), x], axis=-1)
        output, state = self.lstm(x, initial_state=hidden)
        # output = tf.reshape(output, (-1, output.shape[2]))
        y = self.fc(output)

        return y, state


decoder_inputs = Input(shape=(None, num_decoder_tokens))
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder = Decoder(num_decoder_tokens, latent_dim, batch_size)
decoder_outputs, _ = decoder(decoder_inputs, hidden=state)


print(f"decoder_inputs: {decoder_inputs.shape}")
print(f"state: {state.shape}")
print(f"decoder_outputs: {decoder_outputs.shape}")

decoder_inputs: (None, None, 3223)
state: (None, 128)
decoder_outputs: (None, None, 3223)


In [28]:
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
model.summary()

Model: "model_5"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            [(None, None, 2185)] 0                                            
__________________________________________________________________________________________________
input_12 (InputLayer)           [(None, None, 3223)] 0                                            
__________________________________________________________________________________________________
encoder_2 (Encoder)             ((None, None, 128),  888960      input_6[0][0]                    
__________________________________________________________________________________________________
decoder_6 (Decoder)             ((None, None, 3223), 1703319     input_12[0][0]                   
                                                                 encoder_2[0][1]            

In [29]:
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
        batch_size=batch_size,
        epochs=epochs,
        validation_split=0.2)

Epoch 1/35
100/100 [==============================] - 59s 394ms/step - loss: 3.7395 - val_loss: 4.1368
Epoch 2/35
100/100 [==============================] - 23s 225ms/step - loss: 4.2574 - val_loss: 4.5787
Epoch 3/35
100/100 [==============================] - 23s 233ms/step - loss: 4.1552 - val_loss: 3.1784
Epoch 4/35
100/100 [==============================] - 21s 208ms/step - loss: 4.1904 - val_loss: 4.5141
Epoch 5/35
100/100 [==============================] - 21s 207ms/step - loss: 4.0459 - val_loss: 3.1513
Epoch 6/35
100/100 [==============================] - 20s 202ms/step - loss: 3.8644 - val_loss: 3.2849
Epoch 7/35
100/100 [==============================] - 21s 211ms/step - loss: 4.0730 - val_loss: 4.6415
Epoch 8/35
100/100 [==============================] - 22s 218ms/step - loss: 4.1960 - val_loss: 3.6445
Epoch 9/35
100/100 [==============================] - 20s 204ms/step - loss: 4.1779 - val_loss: 4.6468
Epoch 10/35
100/100 [==============================] - 21s 207ms/step - l

In [30]:
# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())

In [ ]:
# encoder_model = Model(encoder_inputs, encoder_states)

# decoder_state_input_h = Input(shape=(latent_dim,))
# decoder_state_input_c = Input(shape=(latent_dim,))
# decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
# decoder_outputs, state_h, state_c = decoder(
#     decoder_inputs, hidden=decoder_states_inputs)
# decoder_states = [state_h, state_c]
# decoder_model = Model(
#     [decoder_inputs] + decoder_states_inputs,
#     [decoder_outputs] + decoder_states)

In [42]:
encoder_model = Model(encoder_inputs, state)

decoder_state_input = Input(shape=(latent_dim,))
decoder_outputs, decoder_state = decoder(
    decoder_inputs, hidden=decoder_state_input)

decoder_model = Model(
    [decoder_inputs] + [decoder_state_input],
    [decoder_outputs] + [decoder_state])

In [44]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index['\t']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, the_decoder_state = decoder_model.predict(
            [target_seq] + [states_value])

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = the_decoder_state

    return decoded_sentence


for seq_index in range(10):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', input_texts[seq_index])
    print('Decoded sentence:', decoded_sentence)

-
Input sentence: ['a', 'string', 'of', 'disasters', 'struck', 'the', 'region']
Decoded sentence: 我我我我我我我我我我我我我我我我我我我我我我我
-
Input sentence: ['tom', 'has', 'been', 'under', 'a', 'great', 'deal', 'of', 'strain', 'lately']
Decoded sentence: 。。我我我我我我我我我我我我我我我我我我我我我
-
Input sentence: ['tom', 'was', 'questioned', 'by', 'the', 'police']
Decoded sentence: 。。我我我我我我我我我我我我我我我我我我我我我
-
Input sentence: ['i', "can't", 'keep', 'up', 'with', 'tom']
Decoded sentence: 我我我我我我我我我我我我我我我我我我我我我我我
-
Input sentence: ['do', 'what', 'you', 'think', 'is', 'right']
Decoded sentence: 。。。我我我我我我我我我我我我我我我我我我我我
-
Input sentence: ['this', 'is', 'all', 'i', 'know']
Decoded sentence: 。。我我我我我我我我我我我我我我我我我我我我我
-
Input sentence: ['what', 'tom', 'said', "doesn't", 'apply', 'in', 'this', 'situation']
Decoded sentence: 。。我我我我我我我我我我我我我我我我我我我我我
-
Input sentence: ['he', 'differs', 'from', 'his', 'brother']
Decoded sentence: 。。我我我我我我我我我我我我我我我我我我我我我
-
Input sentence: ['i', 'prefer', 'milk', 'to', 'juice']
Decoded sentence: 我我我我我我我我我我我